In [21]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from time import time
import numpy as np
import datetime
from thundersvm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import roc_curve
%matplotlib inline

In [3]:
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
feature_name = ["emb","PSNP","PCP","DBPF"]
network_name = ["SVM"]

In [68]:
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        pred_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":

                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                os.environ['CUDA_VISIBLE_DEVICES'] = '0'

                model = SVC(kernel = "rbf",gamma="auto", degree = 1,tol =1e-2, cache_size=7000)

                trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_3,pred_4]).T)
                
                #trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_4]).T)

                clf =model.fit(trainData, true)


                th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),true.reshape(-1))

                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_4.iloc[:,i]]).T)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),testLabel.reshape(-1),th)

                    Auc.append(auc)
#                     th_All[row][-1] = th
#                     Acc[row][-1] = acc
#                     Mcc[row][-1] = mcc
#                     Auc[row][-1] = auc

#                     if not os.path.exists(model_dir):
#                         os.makedirs(model_dir)
#                     model_dir = model_dir+"svm.pkl"
#                     with open(model_dir,"wb") as f:
#                         pickle.dump(clf,f)

In [69]:
np.mean(Auc)

0.7748379653393918

In [65]:
#软投票
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        pred_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold0"+".csv").iloc[:,10]
        
        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],test_Blastn]).T)
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(np.mean(testData,axis=1),testLabel.reshape(-1),th)
                    Auc.append(auc)

In [63]:
np.mean(Auc)

0.8100471408726394

In [66]:
np.mean(Auc)

0.8136897635576664

In [52]:
#测试其中一个
Auc_emb = []
Auc_PSNP = []
Auc_PCP = []
Auc_DBPF = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
#         true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
#         pred_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
#         pred_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
#         pred_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
#         pred_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":

#                 device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
#                 os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#                 model = SVC(kernel = "rbf",gamma="auto", degree = 1,tol =1e-3, cache_size=7000)

#                 trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_3,pred_4]).T)

#                 clf =model.fit(trainData, true)


#                 th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),true.reshape(-1))

                for i in range(5):
                    #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(np.array(test_1.iloc[:,i]),testLabel.reshape(-1),0.5)
                    Auc_emb.append(auc)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(np.array(test_2.iloc[:,i]),testLabel.reshape(-1),0.5)
                    Auc_PSNP.append(auc)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(np.array(test_3.iloc[:,i]),testLabel.reshape(-1),0.5)
                    Auc_PCP.append(auc)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(np.array(test_4.iloc[:,i]),testLabel.reshape(-1),0.5)
                    Auc_DBPF.append(auc)

In [53]:
np.mean(Auc_emb),np.mean(Auc_PSNP),np.mean(Auc_PCP),np.mean(Auc_DBPF)

(0.7808653746493575,
 0.7622671770700092,
 0.7427130636016642,
 0.7703611159484904)

In [54]:
np.mean(Auc)

0.8100471408726394

In [63]:
#原来的训练数据,五折训练
#3 emb 4 PSNP 8 PCP 9 DBPF 10 Blastn 11 True
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    model = SVC(kernel = "rbf",gamma="auto", degree = 1,tol =1e-3, cache_size=7000)

                    train_emb = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    train_true = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]
                    
                    trainData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF]).T)

                    clf =model.fit(trainData, train_true)


                    th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),true.reshape(-1))

                 
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),testLabel.reshape(-1),th)

                    Auc.append(auc)


In [64]:
np.mean(Auc)

0.7959004807338254

In [65]:
Auc

[0.8579215076695079,
 0.850964374100167,
 0.8573112399955426,
 0.8544156156069839,
 0.8356099108215869,
 0.8393589658646785,
 0.8486493899234733,
 0.849070975896747,
 0.8422452845755447,
 0.8312373584511075,
 0.826489831351755,
 0.8195944778950423,
 0.830503301459644,
 0.8165823957687101,
 0.8234724655858213,
 0.8786070877969665,
 0.8774028469656994,
 0.8700002602622128,
 0.8821234774012887,
 0.8765419689159234,
 0.7915783016329476,
 0.7040151024039625,
 0.785437260836372,
 0.7851582000353388,
 0.7787115079210187,
 0.8030089536054803,
 0.8108595771397213,
 0.809587617589149,
 0.8110930348514201,
 0.791219611371491,
 0.8040910193575074,
 0.7854811561918835,
 0.8113388882707337,
 0.8041874032520043,
 0.8087648123103933,
 0.8033166146141414,
 0.7836832069289508,
 0.7970610942146938,
 0.7761340750912396,
 0.7919092587586462,
 0.7428543830948622,
 0.7351535975555095,
 0.7367483089669217,
 0.7369592693990992,
 0.7365280951160369,
 0.8112841338950767,
 0.7895858940589406,
 0.7899705002010995,

In [ ]:
#原来的训练数据,合并训练
#3 emb 4 PSNP 8 PCP 9 DBPF 10 Blastn 11 True
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                model = SVC(kernel = "rbf",gamma="auto", degree = 1,tol =1e-3, cache_size=7000)
                
                for i in range(5):
                    train_emb = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    train_true = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]
                    
                    trainData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF]).T)

                    clf =model.fit(trainData, train_true)


                    th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),true.reshape(-1))

                 
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),testLabel.reshape(-1),th)

                    Auc.append(auc)


In [41]:
#对TestData做五折交叉训练
Auc = []
Auc_train = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    kf = KFold(5,True,10)
                    for j,[train_index, test_index] in enumerate(kf.split(testData)):
                        
                        X_train = testData[train_index]
                        X_test = testData[test_index]
                        Y_train = testLabel[train_index]
                        Y_test = testLabel[test_index]
                        
                        model = LogisticRegression()
                        clf =model.fit(X_train, Y_train)
                        th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(X_train).reshape(-1),Y_train.reshape(-1))
                        Auc_train.append(auc)
                        _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(X_test).reshape(-1),Y_test.reshape(-1),th)

                        Auc.append(auc)
                        print(auc)
                

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8620221452695482
0.8620748000366617
0.8543591885540518
0.8667032698251429
0.8651290942976424


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8551405241332137
0.859253442844069
0.8576095528057407
0.8624884332308786
0.8668495476893225


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

0.7458789138782875
0.7578169990733659
0.7532124461224317
0.7393477490251684
0.7283135957616816


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7396090028949285
0.7544040000746284
0.7459481457162825
0.7353579357823841
0.7227019901317324


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7364021169610739
0.7555909003277423
0.7501053923580281
0.7291330062128024
0.7263219094818072


In [42]:
np.mean(Auc),np.mean(Auc_train)

(0.8130155846217334, 0.8132218483026112)

In [43]:
#对测试集做五折交叉验证,考虑Blastn
Auc = []
Auc_train = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],test_Blastn]).T)
                    kf = KFold(5,True,10)
                    for j,[train_index, test_index] in enumerate(kf.split(testData)):
                        
                        X_train = testData[train_index]
                        X_test = testData[test_index]
                        Y_train = testLabel[train_index]
                        Y_test = testLabel[test_index]
                        
                        model = LogisticRegression()
                        clf =model.fit(X_train, Y_train)
                        th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(X_train).reshape(-1),Y_train.reshape(-1))
                        Auc_train.append(auc)
                        _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(X_test).reshape(-1),Y_test.reshape(-1),th)

                        Auc.append(auc)
                        print(auc)
                

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8671857552049935
0.8661851290476588
0.8579759211182683
0.8699273410199967
0.8700882796047636


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8618024881364627
0.8639985109622454
0.8608261365991083
0.8661194152388869
0.8724577622149882


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

0.7490780319458163
0.7599970770598954
0.7563981555404671
0.7386139044882678
0.7306678025227197


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7428736817542452
0.7572227715691213
0.7498106668258433
0.7344676828547796
0.724647972714582


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.740201846452897
0.7577977201067184
0.7534922488428604
0.7289641597791005
0.7280113917242107


In [44]:
np.mean(Auc),np.mean(Auc_train)

(0.8165320383582648, 0.816774182040436)

In [19]:
#原来的老数据,测试集软投票
#3 emb 4 PSNP 8 PCP 9 DBPF 10 Blastn 11 True
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        
        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    model = SVC(kernel = "rbf",gamma="auto", degree = 1,tol =1e-3, cache_size=7000)

                    train_emb = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    train_true = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]
                    
                    trainData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF,train_Blastn]).T)

                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(np.mean(trainData,axis=1),np.array(train_true).reshape(-1),0.5)
                    
                    Auc.append(auc)


In [20]:
np.mean(Auc)

0.789622520289296

In [60]:
#Logistic回归模型,训练新数据测试新数据
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        pred_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":

                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                os.environ['CUDA_VISIBLE_DEVICES'] = '0'

                #model = SVC(kernel = "rbf",gamma="auto", degree = 1,tol =1e-2, cache_size=7000)
                model = LogisticRegression()
                trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_3,pred_4]).T)
                
                #trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_4]).T)

                clf =model.fit(trainData, true)
                


                th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),true.reshape(-1))

                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_4.iloc[:,i]]).T)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),testLabel.reshape(-1),0.5)

                    Auc.append(auc)
                    print(auc)

0.8484179201957963
0.8472636009982235
0.85273963216949
0.8510226774889623
0.8480091551762864
0.8423964506104639
0.8496328719889563
0.8460252658960482
0.8467391666659063
0.8496388599526795
0.8256514793128333
0.8307085191862055
0.8336144045231915
0.8276594661381435
0.8276394669072551
0.8608489147563931
0.8556395254363667
0.8592765403985743
0.864151853475483
0.8553753174411832
0.8049649289150271
0.8011846010470931
0.7990566432699646
0.8027379395573367
0.7975193513928421
0.8046548535058392
0.8090087743199125
0.8071737654769606
0.8067876984638827
0.8070351613649078
0.8110991522949735
0.808637063823916
0.8019605552747374
0.8114580021108114
0.8093303505150297
0.801177022763125
0.7987475199220355
0.7969059846408193
0.7956874834346539
0.7967870133689283
0.7407373643618853
0.7449255725932302
0.7379138456510493
0.7383522300111488
0.7415534166438762
0.783424158999434
0.795901247664884
0.7773837753944911
0.804595129764289
0.785375858241669
0.7508207622638421
0.7511429007990599
0.7442079680506264
0.

In [61]:
np.mean(Auc)

0.8010482484153975

In [36]:
#Logistic回归模型,训练老数据测试新数据
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                
                train_emb = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                train_PSNP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                train_PCP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                train_DBPF = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                train_Blastn = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                train_true = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]

                trainData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF]).T)
                model = LogisticRegression()
                
                #trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_4]).T)

                clf =model.fit(trainData, train_true)
                


                th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),np.array(train_true).reshape(-1))

                
                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_4.iloc[:,i]]).T)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),testLabel.reshape(-1),0.5)

                    Auc.append(auc)
                    print(auc)

0.853879853073066
0.8518293311148553
0.8549919423735072
0.852367716984895
0.854524656778965
0.8415654184954792
0.844083821636022
0.8406232429990911
0.8421246538775308
0.8438044743867883
0.8263467364180841
0.8303721129094278
0.8313709391263295
0.8297635482489888
0.82742279401715
0.8818201503614116
0.8797919783519234
0.8801888284059347
0.8824119380084903
0.8794652755405612
0.7881894448539697
0.7915300168172623
0.7886352937572049
0.7909506739296077
0.7871990671739646
0.8080168168522308
0.8116654408785741
0.8107212003652142
0.8057853791901719
0.807627100473288
0.8151750305786177
0.8139809932060138
0.8113390577810196
0.8141478190024793
0.8157990913446096
0.8057233074828731
0.8057783650242878
0.8055715131070923
0.8034708579493643
0.8057024883912128
0.7407352626390242
0.7432817334099412
0.7390032094767618
0.7407800118216072
0.7408334306109922
0.8069114431462838
0.8159150813308644
0.8052581845158662
0.816190691075942
0.8073631824324997
0.7412429118516423
0.7408884138083349
0.738581127752773
0.

In [37]:
np.mean(Auc)

0.8041950611815216

In [28]:
#Logistic回归模型,训练新数据测试集测试新数据
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                
                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    kf = KFold(5,True,10)
                    for j,[train_index, test_index] in enumerate(kf.split(testData)):
                        
                        X_train = testData[train_index]
                        X_test = testData[test_index]
                        Y_train = testLabel[train_index]
                        Y_test = testLabel[test_index]
                        
                        model = LogisticRegression()
                        clf =model.fit(X_train, Y_train)
                        th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(X_train).reshape(-1),Y_train.reshape(-1))

                        _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(X_test).reshape(-1),Y_test.reshape(-1),0.5)

                        Auc.append(auc)
                        print(auc)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8620221452695482
0.8620748000366617
0.8543591885540518
0.8667032698251429
0.8651290942976424


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8551405241332137
0.859253442844069
0.8576095528057407
0.8624884332308786
0.8668495476893225


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

0.7458789138782875
0.7578169990733659
0.7532124461224317
0.7393477490251684
0.7283135957616816


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7396090028949285
0.7544040000746284
0.7459481457162825
0.7353579357823841
0.7227019901317324


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7364021169610739
0.7555909003277423
0.7501053923580281
0.7291330062128024
0.7263219094818072


In [29]:
np.mean(Auc)

0.8130155846217334

In [38]:
#Logistic回归模型,训练老数据测试新数据 带Blastn的
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                
                train_emb = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                train_PSNP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                train_PCP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                train_DBPF = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                train_Blastn = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                train_true = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]

                trainData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF,train_Blastn]).T)
                model = LogisticRegression()
                
                #trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_4]).T)
                test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]

                clf =model.fit(trainData, train_true)
                


                th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),np.array(train_true).reshape(-1))

                
                for i in range(5):
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],test_Blastn]).T)
                    #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_4.iloc[:,i]]).T)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),testLabel.reshape(-1),0.5)

                    Auc.append(auc)
                    print(auc)

0.8560165928369702
0.8549341263711492
0.8567742410391892
0.85479876975181
0.8563786474243811
0.8463623401751154
0.8482686051482687
0.8453153862405149
0.8471737147230604
0.8481588877326789
0.8372536241037136
0.8397824642609815
0.8402918478739143
0.8392932545447088
0.8376430414422783
0.8861194589208381
0.8853784888139659
0.885430820251238
0.8868025575525245
0.8852491149789433
0.7988541528338067
0.8014217460233545
0.798971601813378
0.8001709651116334
0.7979262927061805
0.8156548438858715
0.8181550543525784
0.8172847105066943
0.8140038946989718
0.8151770880510859
0.8169310671962159
0.8156461411758321
0.8134439292700135
0.8159575246522267
0.8174442232631183
0.805645022999987
0.8056635693163376
0.8055837579043028
0.8038276014537094
0.8056096452261756
0.7435807618681152
0.7456622848372167
0.7419955665907683
0.7438228102842787
0.7438477390526583
0.8083961292124698
0.816750467143124
0.8071102836021478
0.8167996474690266
0.8087656613848196
0.743343608726359
0.7429415874064831
0.74092704414608
0.

In [39]:
np.mean(Auc)

0.8082601679415221

In [52]:
##老数据带Blastn,训练加结果
Auc = []
Auc_train = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    train_emb = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    trainLabel = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]

                    trainData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF,train_Blastn]).T)
                    model = LogisticRegression()
                    clf =model.fit(trainData, trainLabel)
                    th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),np.array(trainLabel).reshape(-1))
                    Auc_train.append(auc)
                    
                    train_emb = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    testLabel = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]
                    testData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF,train_Blastn]).T)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),
                                                                          np.array(testLabel).reshape(-1),th)

                    Auc.append(auc)
                    print(auc)

0.8423214443797283
0.8389400564652065
0.8453946296328401
0.843535187099838
0.8431659568159677
0.8272349920253272
0.8348317893847836
0.834116910653697
0.8306970840436004
0.8237673381940338
0.8199890769848416
0.8182183153888025
0.8221137696567768
0.8216966677933579
0.8195727320064343
0.8667341488455138
0.8699701288329839
0.8700296344200722
0.8697360921234172
0.8665504507538258
0.7889946422709992
0.7791882736630555
0.7830557955648454
0.7807012863573802
0.7814956345895812
0.7986500074351344
0.8021777716431208
0.800844640345063
0.804315342829087
0.7984541501569754
0.7945354166557983
0.7936665276866823
0.7949687022435039
0.7955892759407575
0.793226707304486
0.776054522082599
0.7755260695226933
0.7764910006525214
0.7826230717667094
0.7826793743426608
0.7259780155702051
0.7284714178388807
0.7291777134825725
0.7302016319936396
0.7268001978772073
0.7846142029342582
0.7935000482407019
0.7808401527053118
0.7887079652655751
0.7865400725093934
0.7197513776572193
0.7206118773806481
0.7220629597869039

In [54]:
np.mean(Auc),np.mean(Auc_train)

(0.7894866669876326, 0.8319548321210921)

In [56]:
##老数据带Blastn,训练加结果
Auc = []
Auc_train = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    
                    train_emb = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    testLabel = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]
                    testData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF,train_Blastn]).T)
                    testLabel = np.array(testLabel)
                    kf = KFold(5,True,10)
                    for j,[train_index, test_index] in enumerate(kf.split(testData)):
                        
                        X_train = testData[train_index]
                        X_test = testData[test_index]
                        Y_train = testLabel[train_index]
                        Y_test = testLabel[test_index]
                        
                        model = LogisticRegression()
                        clf =model.fit(X_train, Y_train)
                        th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(X_train).reshape(-1),Y_train.reshape(-1))
                        Auc_train.append(auc)
                        _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(X_test).reshape(-1),Y_test.reshape(-1),th)

                        Auc.append(auc)
                        print(auc)
                

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8423037065538027
0.8470153924233491
0.8388157306785827
0.8418008069799144
0.8475257695986471


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8385904835193024
0.8440694642610419
0.8356679050839246
0.8392623608353046
0.845164839224798


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

0.7320825618547426
0.7434320913947399
0.7344712599081252
0.712419696884892
0.7144209155041241


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7295974660598616
0.7384976087862335
0.7349609146688753
0.7089765356318837
0.7066829997792543


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7247183138634352
0.7377886403353298
0.7284853470424231
0.7050573082831147
0.7053218379726214


In [57]:
np.mean(Auc),np.mean(Auc_train)

(0.7935313302572278, 0.793774302807808)

In [58]:
##老数据不带Blastn,训练加结果
Auc = []
Auc_train = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    train_emb = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    trainLabel = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]

                    trainData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF]).T)
                    model = LogisticRegression()
                    clf =model.fit(trainData, trainLabel)
                    th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(trainData).reshape(-1),np.array(trainLabel).reshape(-1))
                    Auc_train.append(auc)
                    
                    train_emb = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,3]
                    train_PSNP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,4]
                    train_PCP = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,8]
                    train_DBPF = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,9]
                    train_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,10]
                    testLabel = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,11]
                    testData = np.array(pd.DataFrame([train_emb,train_PSNP,train_PCP,train_DBPF]).T)
                    
                    _,_,_,_,_,_, _,_, auc = metricsCal.get_test_metrics(clf.decision_function(testData).reshape(-1),
                                                                          np.array(testLabel).reshape(-1),th)

                    Auc.append(auc)
                    print(auc)

0.8331754720015286
0.8304090392214094
0.8380536698384282
0.8344608280354912
0.8337433883592849
0.8199301789224868
0.8292484612176172
0.8288790595833041
0.8240830776206024
0.8149734671196678
0.808199079156228
0.8008570616541937
0.8099504237406182
0.8099964481715274
0.8065908440609896
0.8574692524906554
0.8619984526953101
0.8618566835236601
0.8602979799068803
0.8567218655053365
0.7784764088701268
0.7630944743497817
0.7722379777924497
0.7706268209264977
0.7689639452165242
0.7895314506179059
0.7951942613685883
0.7921863822321399
0.7971483352597997
0.7875234020072996
0.7893490142201971
0.7874882998119709
0.7885316840531407
0.7898312363896213
0.78734844344793
0.7714147305709145
0.7693388703934376
0.7709929986128847
0.7773060401952001
0.7779384531389056
0.7167714769585933
0.7203828498363678
0.7223230903242546
0.7230927587503261
0.7190056959316687
0.7756695011727509
0.7869190009228708
0.7740554668510409
0.7820931714476941
0.779785894450784
0.7133794006309302
0.7141761916021857
0.71627658441686

In [59]:
np.mean(Auc),np.mean(Auc_train)

(0.7814551804724797, 0.8272232907546385)

# 开始生成集成学习的结果

In [71]:
#软投票
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]

Param_SoftVoting = np.zeros(60*6).reshape(60,6)

row_name = []
rows = -1
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        trainLabel = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1]
        
        pred_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        pred_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,2]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold0"+".csv").iloc[:,10]
        
        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    rows+=1
                    row_name.append([type_name[typei]+"_"+cell_name[celli]+"_Kfold_"+str(i)])
                    trainData = np.array(pd.DataFrame([pred_1,pred_2,pred_3,pred_4]).T)
                    testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics(np.mean(trainData,axis=1),trainLabel)
                    _,_,_,_,sen,spe, acc,mcc, auc = metricsCal.get_test_metrics(np.mean(testData,axis=1),testLabel.reshape(-1),th)
                    Param_SoftVoting[typei*30+celli*5+i][0] = sen
                    Param_SoftVoting[typei*30+celli*5+i][1] = spe
                    Param_SoftVoting[typei*30+celli*5+i][2] = acc
                    Param_SoftVoting[typei*30+celli*5+i][3] = mcc
                    Param_SoftVoting[typei*30+celli*5+i][4] = auc
                    Param_SoftVoting[typei*30+celli*5+i][5] = th

In [79]:
header_name = ["name",'Sen', 'Spe', 'Acc', 'mcc', 'AUC','th']
pd.DataFrame(np.append(np.array(row_name).reshape(-1,1),Param_SoftVoting,axis=1)).to_csv("Table/Soft_Voting_result.csv",header=header_name,index=0)

In [81]:
def para4(x1, x2, x3, x4, label, th=0.5):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))

    max6 = max(max1,max2,max3,max4)
    min6 = min(min1,min2,min3,min4)
    
    pred =[]
    for j in range(len(x1)):
        temp1l = 0
        if x1[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x2[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x3[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x4[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        
        if temp1l>0:
            pred.append(1)
        elif temp1l<0:
            pred.append(0)
        elif np.mean([x1[j],x2[j],x3[j],x4[j]])>th:
            pred.append(1)
        else:
            pred.append(0)
    TN, FN, FP, TP, Sen, Spe, Acc, mcc, AUC = metricsCal.get_test_metrics(np.array(pred),label,th)
    return Sen, Spe, Acc, mcc, AUC

In [82]:
def roc4(x1, x2, x3, x4, label, iter = 500, low = 0.01 , high = 0.99):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))
    
    max6 = max(max1,max2,max3,max4)
    min6 = min(min1,min2,min3,min4)
    
    TPR = []
    FPR = []
    
    for i in range(iter+1):
        if i == 0 or (i/iter > low and i/iter < high) or i == iter:
            bool_ = min6+i*(max6-min6)/iter
            tp = 0
            tn = 0
            for j in range(len(x1)):
                temp1l = 0
                if x1[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x2[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x3[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x4[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                
                if temp1l>0 and label[j]==1:
                    tp += 1
                elif temp1l<0 and label[j]==0:
                    tn += 1
                elif np.mean([x1[j],x2[j],x3[j],x4[j]])>bool_ and label[j]==1:
                    tp += 1
                elif np.mean([x1[j],x2[j],x3[j],x4[j]])<bool_ and label[j]==0:
                    tn += 1
            tpr_ = tp/num_pos
            fpr_ = 1- tn/num_neg
            TPR.append(tpr_)
            FPR.append(fpr_)
    area = 0
    for i in range(len(TPR)-1):
        temp = (-FPR[i+1]+FPR[i])*(TPR[i]+TPR[i+1])/2
        area += temp 
    return area

In [83]:
#硬投票
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]

Param_HardVoting = np.zeros(60*6).reshape(60,6)

row_name = []
rows = -1
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold0"+".csv").iloc[:,10]
        
        for networki in range(len(network_name)):
            if network_name[networki] == "SVM":
                for i in range(5):
                    rows+=1
                    row_name.append([type_name[typei]+"_"+cell_name[celli]+"_Kfold_"+str(i)])
                    #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)
                    
                    #_,_,_,_,sen,spe, acc,mcc, _ = metricsCal.get_test_metrics(np.mean(testData,axis=1),testLabel.reshape(-1),th)
                    sen,spe,acc,mcc,_ = para4(test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],testLabel)
                    auc = roc4(test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],testLabel)
                    
                    Param_HardVoting[typei*30+celli*5+i][0] = sen
                    Param_HardVoting[typei*30+celli*5+i][1] = spe
                    Param_HardVoting[typei*30+celli*5+i][2] = acc
                    Param_HardVoting[typei*30+celli*5+i][3] = mcc
                    Param_HardVoting[typei*30+celli*5+i][4] = auc
                    Param_HardVoting[typei*30+celli*5+i][5] = 0.5

In [84]:
header_name = ["name",'Sen', 'Spe', 'Acc', 'mcc', 'AUC','th']
pd.DataFrame(np.append(np.array(row_name).reshape(-1,1),Param_SoftVoting,axis=1)).to_csv("Table/Hard_Voting_result.csv",header=header_name,index=0)

In [85]:
Param_HardVoting[:,-2]

array([0.87899331, 0.87555326, 0.87962725, 0.87812886, 0.8780614 ,
       0.86534171, 0.86943568, 0.86515172, 0.86741532, 0.87185431,
       0.84998411, 0.85127399, 0.85207498, 0.85118329, 0.85010581,
       0.90212983, 0.90165044, 0.90177258, 0.90313774, 0.90280707,
       0.81390413, 0.81521987, 0.81460369, 0.81391513, 0.8120204 ,
       0.83093303, 0.8357769 , 0.83329443, 0.83157517, 0.83147743,
       0.84680537, 0.84378003, 0.84170046, 0.84839925, 0.84547885,
       0.82819033, 0.83045691, 0.82937324, 0.82991855, 0.82937095,
       0.77655944, 0.77579946, 0.77320078, 0.77370929, 0.7793263 ,
       0.835113  , 0.84491586, 0.83615444, 0.843857  , 0.83694233,
       0.76812687, 0.77047337, 0.7686218 , 0.76876583, 0.76514344,
       0.76170644, 0.76456007, 0.77151011, 0.7657982 , 0.76165394])

In [86]:
np.mean(Param_HardVoting[:,-2])

0.8307302454477582